# Day 2 of AI Academy 2022 - NLP Track

Hello! Welcome to the second day in your AI academy 2022, NLP track. In this lab we'll go over the usage of the HuggingFace's `transformers` library to train a BERT-based model for our sentiment analysis task. BERT is a transfromer-based model, a deep learning model, which requires more computation than the traditional models we worked with yesterday. Deep learning models typically require a GPU to train and run effciently, if you're local machine is not powered with a GPU to use, we recommend that you run this notebook in Google's Colabortatory (or Colab for short), which is a free notebook runtime that can assign you a GPU for a limited amount of time (usually 12 hours). If you wish to run this in Colab, simply click the following badge.